# Machine Translation

In [1]:
!nvidia-smi
!lscpu

Wed Jul  7 10:10:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0
!pip install sacrebleu==1.5.1

     |████████████████████████████████| 317kB 9.8MB/s 
     |████████████████████████████████| 1.4MB 44.4MB/s 
     |████████████████████████████████| 235kB 51.6MB/s 
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394489 sha256=c7d306e705b23e810c1b8f5650e6ae593afaf9678cc11f22bd90de73004ad346
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154157 sha256=952e56ae415c6565b06ab1c79822fa513a7eb40a0ba0cf23b2e5ebede0bd73cd
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 51kB 4.4MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=f91afd9704426ef

In [4]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
# Import required packages
import os
import gc
import re
import hazm
import time
import json
import sacrebleu
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import MT5Config, MT5ForConditionalGeneration, MT5Tokenizer

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [6]:
class MachineTranslationDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Machine Translation. """

    def __init__(self, original_text, translated_text, tokenizer, max_length):
        self.original_text = original_text
        self.translated_text = translated_text
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.original_text)

    def __getitem__(self, item):
        encoding = self.tokenizer(
            self.original_text[item],
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        inputs = {
            'original': self.original_text[item],
            'translated': self.translated_text[item],
            'input_ids': encoding.input_ids.flatten(),
            'attention_mask': encoding.attention_mask.flatten()
        }
        return inputs


class MachineTranslation:
    def __init__(self, model_name, model_type):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        if model_type.lower() == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(model_name)
            self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
            self.config = MT5Config.from_pretrained(self.model_name)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
            self.id2label = self.config.id2label
            self.label2id = self.config.label2id

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "mizan":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t", names=['original', 'translation'], header=None)
            original, translation = data['original'].values.tolist(), data['translation'].values.tolist()
            print(f'test part:\n #original: {len(original)}, #translation: {len(translation)}')
            return original, translation
        if dataset_name.lower() == "combined":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t", names=['original', 'translation', 'source'], header=None)
            if 'source' in kwargs:
                data = data[data['source'] == kwargs['source']]
                data = data[['original', 'translation']]
            else:
                data = data[['original', 'translation']]
            original, translation = data['original'].values.tolist(), data['translation'].values.tolist()
            print(f'test part:\n #original: {len(original)}, #translation: {len(translation)}')
            return original, translation

    def load_dataset_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "quran" or dataset_name.lower() == "bible":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            original, translation = [], []
            with open(dataset_file, encoding="utf8") as infile:
                for line in infile:
                    parts = line.strip().split('\t')
                    original.append(parts[0])
                    translation.append(parts[1])
            print(f'all data:\n #original: {len(original)}, #translation: {len(translation)}')

            _, original_test, _, translation_test = train_test_split(original, translation, test_size=0.1,
                                                                     random_state=1)
            print(f'test part:\n #original: {len(original_test)}, #translation: {len(translation_test)}')
            return original, translation, original_test, translation_test

    def mt5_machine_translation_inference(self, input_text, device):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        tokenized_batch = self.tokenizer(
            input_text,
            padding=True,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        input_ids = tokenized_batch.input_ids.to(device)
        attention_mask = tokenized_batch.attention_mask.to(device)
        outputs = self.model.generate(input_ids=input_ids,
                                      attention_mask=attention_mask)
        predictions = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return predictions

    def mt5_evaluation(self, input_text, translated_text, device, max_length, split_reference=None, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(input_text) != len(translated_text):
            print('length of inputs and its translations is not equal!!')
            return

        dataset = MachineTranslationDataset(original_text=input_text, translated_text=translated_text,
                                            tokenizer=self.tokenizer, max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#original_text:{len(input_text)}, #translated_text:{len(translated_text)}')
        print("#batch:", len(data_loader))

        if split_reference is None:
            max_num_ref = 1
        else:
            max_num_ref = 0
            for ref in translated_text:
                max_num_ref = max(max_num_ref, len(ref.split(split_reference)))
        print("#maximum_translation_reference:", max_num_ref)

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        total_time = 0
        output_predictions = []
        golden_translations, predicted_translations = [[] for _ in range(max_num_ref)], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model.generate(input_ids=b_input_ids, attention_mask=b_attention_mask)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')

            b_predictions = self.tokenizer.batch_decode(b_outputs, skip_special_tokens=True)

            for i in range(len(b_input_ids)):
                if split_reference is None:
                    sample_golden_translation = [batch['translated'][i]]
                else:
                    sample_golden_translation = batch['translated'][i].split(split_reference)
                sample_generated_translation = b_predictions[i]
                bleu_score = sacrebleu.corpus_bleu(sys_stream=[sample_generated_translation],
                                                   ref_streams=[[g] for g in sample_golden_translation]).score
                output_predictions.append((batch['original'][i], batch['translated'][i], b_predictions[i], bleu_score))

                for j in range(max_num_ref):
                    try:
                        golden_translations[j].append(sample_golden_translation[j])
                    except:
                        golden_translations[j].append('')
                predicted_translations.append(sample_generated_translation)

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(input_text))

        # evaluate
        print("BLEU Score: {}".format(sacrebleu.corpus_bleu(
            sys_stream=predicted_translations, ref_streams=golden_translations).score))
        return output_predictions


In [7]:
model_name='persiannlp/mt5-base-parsinlu-translation_en_fa'
mt_model = MachineTranslation(model_name=model_name, model_type="mt5")
print(mt_model.config)


MT5Config {
  "_name_or_path": "/home/patrick/hugging_face/t5/mt5-base",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.7.0",
  "use_cache": true,
  "vocab_size": 250112
}



## Sample Inference

In [ ]:
input_list = [
  "Praise be to Allah, the Cherisher and Sustainer of the worlds;",
  "shrouds herself in white and walks penitentially disguised as brotherly love through factories and parliaments; offers help, but desires power;",
  "He thanked all fellow bloggers and organizations that showed support.",
  "Races are held between April and December at the Veliefendi Hippodrome near Bakerky, 15 km (9 miles) west of Istanbul.",
  "I want to pursue PhD in Computer Science about social network,what is the open problem in social networks?"
]
mt_model.mt5_machine_translation_inference(input_list, device)

['خدا را شکر که عامل خطرناک و محافظ دنیاست.',
 'خود را سفید می کند و به شکل برادرانه ای در کارخانه ها و',
 'او از تمامی همکاران و سازمان هایی که از او حمایت می کردند تشکر',
 'برگزاری مسابقات بین آوریل تا دسامبر در هیپوگریم والی',
 'من می خواهم تحصیل دکترای علوم کامپیوتری را در مورد شب']

## Mizan Dataset


In [ ]:
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/mizan/mizan_test_en_fa.tsv
!ls

--2021-07-03 20:38:28--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/mizan/mizan_test_en_fa.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2296459 (2.2M) [text/plain]
Saving to: ‘mizan_test_en_fa.tsv’

mizan_test_en_fa.ts 100%[===================>]   2.19M  --.-KB/s    in 0.04s   

2021-07-03 20:38:29 (53.8 MB/s) - ‘mizan_test_en_fa.tsv’ saved [2296459/2296459]

adc.json  mizan_test_en_fa.tsv	sample_data


In [ ]:
mizan_original_text, mizan_translated_text = mt_model.load_dataset_test_file(dataset_name="mizan", dataset_file="mizan_test_en_fa.tsv")
print(mizan_original_text[:5])
print(mizan_translated_text[:5])
print(len(mizan_original_text))
print(len(mizan_translated_text))

test part:
 #original: 10000, #translation: 10000
['a sure sign with him of mental agitation.', ';Cesare, you are worried about this affair,; she said at last.', ';I am very sorry you feel so despondent over it;', 'but I could decide only as seemed right to me.;', ';It is not the affair,; he answered, sullenly; ;I know nothing about it,']
['این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.', 'عاقبت گفت: سزار! تو از این موضوع نگران شده\u200cای؛', 'بسیار متأسفم که نسبت به آن احساس بی اعتمادی می\u200cکنی؛', 'اما من فقط بدان علت نتوانستم تصمیم بگیرم که آن کار به نظرم صحیح رسید.', 'مارتینی، با اخم، پاسخ داد: به خاطر آن موضوع نیست، من در آن باره چیزی نمی\u200cدانم؛']
10000
10000


In [ ]:
mt_model.mt5_machine_translation_inference(mizan_original_text[:5], device)

['نشانه قطعی اضطراب ذهنی او.',
 'سرانجام گفت: «سپار» تو نگران این موضوع هستی.',
 'متاسفم که این قدر از این موضوع ناراحتی',
 'اما من فقط وقتی فکر می کنم که درست به نظر می رس',
 'او با لحنی عبوس پاسخ داد: این قضیه نیست. من چیزی']

In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 20:38:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    33W /  70W |   3412MiB / 15109MiB |     56%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(mizan_original_text, mizan_translated_text, device, max_length=512, batch_size=128)

#original_text:10000, #translated_text:10000
#batch: 79
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 6.7726440999999795
inference time for step 1: 6.825832088000027
inference time for step 2: 6.856606778000014
inference time for step 3: 6.88191898499997
inference time for step 4: 6.926321956000038
inference time for step 5: 6.94751603200001
inference time for step 6: 6.99135494899997
inference time for step 7: 7.0361592470000005
inference time for step 8: 7.082116010999982
inference time for step 9: 7.12352867300001
inference time for step 10: 7.16242892300005
inference time for step 11: 7.19538578099997
inference time for step 12: 7.217387218999988
inference time for step 13: 7.257835545999967
inference time for step 14: 7.3093792810000195
inference time for step 15: 7.348293216999991
inference time for step 16: 7.40027662
inference time for step 17: 7.435229591999985
inference time for step 18: 7.490272455000024
inference time for step 

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

a sure sign with him of mental agitation.	این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.	نشانه قطعی اضطراب ذهنی او.	1.8110800993753928
;Cesare, you are worried about this affair,; she said at last.	عاقبت گفت: سزار! تو از این موضوع نگران شده‌ای؛	سرانجام گفت: «سپار» تو نگران این موضوع هستی.	11.884631831419354
;I am very sorry you feel so despondent over it;	بسیار متأسفم که نسبت به آن احساس بی اعتمادی می‌کنی؛	متاسفم که این قدر از این موضوع ناراحتی	4.300847718252331
but I could decide only as seemed right to me.;	اما من فقط بدان علت نتوانستم تصمیم بگیرم که آن کار به نظرم صحیح رسید.	اما من فقط وقتی فکر می کنم که درست به نظر می رس	10.372689474549311
;It is not the affair,; he answered, sullenly; ;I know nothing about it,	مارتینی، با اخم، پاسخ داد: به خاطر آن موضوع نیست، من در آن باره چیزی نمی‌دانم؛	او با لحنی عبوس پاسخ داد: این قضیه نیست. من چیزی	10.052623571648706
and it probably is all right, once you have consented to go into it.	هنگامی که تو قبول می‌کنی در کاری شرکت جویی قطع

In [ ]:
output_file_name = "translation_en-fa_mizan_test_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## Quran Dataset

In [ ]:
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/quran/quran_en_fa.tsv
!ls

--2021-07-03 20:48:25--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/quran/quran_en_fa.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16215125 (15M) [text/plain]
Saving to: ‘quran_en_fa.tsv’

quran_en_fa.tsv     100%[===================>]  15.46M  58.8MB/s    in 0.3s    

2021-07-03 20:48:27 (58.8 MB/s) - ‘quran_en_fa.tsv’ saved [16215125/16215125]

adc.json
mizan_test_en_fa.tsv
quran_en_fa.tsv
sample_data
translation_en-fa_mizan_test_persiannlp-mt5-base-parsinlu-translation_en_fa_outputs.txt


In [ ]:
quran_original_all, quran_translated_all, quran_original_test, quran_translated_test = mt_model.load_dataset_file(dataset_name="quran", dataset_file="quran_en_fa.tsv")
print(quran_original_all[0])
print(quran_translated_all[0])
print(quran_original_test[0])
print(quran_translated_test[0])

all data:
 #original: 6236, #translation: 6236
test part:
 #original: 624, #translation: 624
In the name of Allah, Most Gracious, Most Merciful.
به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌
Nay, but man doth transgress all bounds,
این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///حقا که آدمى نافرمانى مى‌کند،///چنین نیست [که مى‌پندارند]، بى‌شک آدمى طغیان مى‌کند///حقا که انسان سرکشى مى‌کند،///این چنین نیست [که انسان سپاس‌گزار باشد]، بى‌گمان آدمى سرکشى مى‌کند.///راستی که انسان سرکش و مغرور می‌شود.///چنین نیست، بی‌گمان انسان سر به طغیان برآورد

In [ ]:
mt_model.mt5_machine_translation_inference(quran_original_test[:5], device)

['نه، اما انسان از تمام محدودیت ها تجاوز می کند.',
 'مصری ها می گویند: "اگر شما دروغ گفته باشید، پس چه',
 'پس هر دو به طرف او بروید و بگوئید: «مسلما ما فر',
 'اما در مورد کسانی که ایمان را رد می کنند و می میرند، هر',
 'دعا می کنم: "خدای من! استخوان های من واقعا ضع']

### test set

In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 20:48:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    42W /  70W |   3400MiB / 15109MiB |     65%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(quran_original_test, quran_translated_test, device, max_length=512, split_reference='///', batch_size=128)

#original_text:624, #translated_text:624
#batch: 5
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 7.493456160999926
inference time for step 1: 7.542606805999981
inference time for step 2: 7.574265013000058
inference time for step 3: 7.616502458000014
inference time for step 4: 6.687872894999941
total inference time: 36.91470333299992
total inference time / #samples: 0.059158178418269106
BLEU Score: 5.561762223201149


In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

Nay, but man doth transgress all bounds,	این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///حقا که آدمى نافرمانى مى‌کند،///چنین نیست [که مى‌پندارند]، بى‌شک آدمى طغیان مى‌کند///حقا که انسان سرکشى مى‌کند،///این چنین نیست [که انسان سپاس‌گزار باشد]، بى‌گمان آدمى سرکشى مى‌کند.///راستی که انسان سرکش و مغرور می‌شود.///چنین نیست، بی‌گمان انسان سر به طغیان برآورد///چنین نیست (که شما می‌پندارید) به یقین انسان طغیان می‌کند،///نه چنین است همانا انسان سرکشی کند///چنان نیست (که انسان سرنهد). همواره انسان بی‌گمان سرکشی می‌کند،	نه، اما انسان از تمام محدودیت ها تجاوز می کند.	4.9323515694897075
(The Egyptians) said: "What then shall be the penalty of this, if ye are (proved) to have lied?"	گماشتگان گفتند: اگر شما دروغگو باشید [و سارق در میان شما یافت شود] کیفرش چیست؟///گماشتگان گفتند: اگر شما دروغگو باشید [و سارق در میان شما یافت شود] کیفرش چیست؟///گفتند: اگر دروغ گفته باشید جزاى دزد چیست؟///گفتند: پس اگر دروغ گفته باشی

In [ ]:
output_file_name = "translation_en-fa_quran_test_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### all dataset

In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 20:49:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    33W /  70W |   9556MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(quran_original_all, quran_translated_all, device, max_length=512, split_reference='///', batch_size=128)

#original_text:6236, #translated_text:6236
#batch: 49
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 7.627237753999907
inference time for step 1: 7.608095945000059
inference time for step 2: 7.597175476000075
inference time for step 3: 7.590397871999926
inference time for step 4: 7.589612347999946
inference time for step 5: 7.590400671999987
inference time for step 6: 7.576841524000088
inference time for step 7: 7.575773560999892
inference time for step 8: 7.577826461999962
inference time for step 9: 7.579442481000001
inference time for step 10: 7.595175626000014
inference time for step 11: 7.5721368200000825
inference time for step 12: 7.583559965999939
inference time for step 13: 7.610168697999939
inference time for step 14: 7.568984548999879
inference time for step 15: 7.594665304000046
inference time for step 16: 7.589999934000161
inference time for step 17: 7.568957746000024
inference time for step 18: 7.544661607999842
inference time

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the name of Allah, Most Gracious, Most Merciful.	به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌	به نام الله، گرامی ترین و قابل ترحیم ترین.	9.980099403873663
Praise be to Allah, the Cherisher and Sustainer of the worlds;	همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///ستایش خدا را که پروردگار جهانیان است.///ستایش خداى را که پروردگار جهانیان است///ستایش خدایى را که پروردگار جهانیان،///سپاس و ستایش مخصوص خداوندى است که پروردگار جهانیان است.///ستایش خدای را که پروردگار جهانیان است.///سپاس خداوند را که پروردگار جهانیان است‌///ستایش مخصوص خداوندی است که پروردگار جهانیان است./

In [ ]:
output_file_name = "translation_en-fa_quran_all_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## Bible Dataset

In [ ]:
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/bible/bible_en_fa.tsv
!ls

--2021-07-03 20:55:54--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/bible/bible_en_fa.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10023337 (9.6M) [text/plain]
Saving to: ‘bible_en_fa.tsv’

bible_en_fa.tsv     100%[===================>]   9.56M  --.-KB/s    in 0.07s   

2021-07-03 20:55:55 (132 MB/s) - ‘bible_en_fa.tsv’ saved [10023337/10023337]

adc.json
bible_en_fa.tsv
mizan_test_en_fa.tsv
quran_en_fa.tsv
sample_data
translation_en-fa_mizan_test_persiannlp-mt5-base-parsinlu-translation_en_fa_outputs.txt
translation_en-fa_quran_all_persiannlp-mt5-base-parsinlu-translation_en_fa_outputs.txt
translation_en-fa_quran_test_persiannlp-mt5-base-parsinlu-translation_en_fa_outputs.txt


In [ ]:
bible_original_all, bible_translated_all, bible_original_test, bible_translated_test = mt_model.load_dataset_file(dataset_name="bible", dataset_file="bible_en_fa.tsv")
print(bible_original_all[0])
print(bible_translated_all[0])
print(bible_original_test[0])
print(bible_translated_test[0])

all data:
 #original: 31020, #translation: 31020
test part:
 #original: 3102, #translation: 3102
In the beginning God created the heaven and the earth.
در ابتدا، خدا آسمانها و زمین را آفرید.
And of the sons of Elizaphan; Shimri, and Jeiel: and of the sons of Asaph; Zechariah, and Mattaniah:
و چون قربانی های سوختنی تمام شد، پادشاه و جمیع حاضرین با وی رکوع کرده، سجده نمودند.


In [ ]:
mt_model.mt5_machine_translation_inference(bible_original_test[:5], device)

['و پسران الیزایات، شمری و جیل: و پسران آزوف',
 'تمام ستون های اطراف دربار با نقره پوشیده خواهند شد؛',
 'و موسی به آنان می گوید: اگر بچه های خر و بچه',
 'من به تو شیر و گوشت داده ام نه گوشت: چون تا کنون',
 'و در پایان هفت روز، کلمه خداوند به گوشم رسید و گفت']

### test set

In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 20:55:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    41W /  70W |   3400MiB / 15109MiB |     57%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(bible_original_test, bible_translated_test, device, max_length=512, batch_size=128)

#original_text:3102, #translated_text:3102
#batch: 25
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 7.423138711000092
inference time for step 1: 7.5987420300000394
inference time for step 2: 7.786922389999972
inference time for step 3: 7.894024307000109
inference time for step 4: 7.811800354999832
inference time for step 5: 7.688571413999853
inference time for step 6: 7.589981161000196
inference time for step 7: 7.520315358000062
inference time for step 8: 7.473210988999881
inference time for step 9: 7.444246577000058
inference time for step 10: 7.447921989000179
inference time for step 11: 7.471120322000161
inference time for step 12: 7.512686597000084
inference time for step 13: 7.559126007000032
inference time for step 14: 7.584485964000123
inference time for step 15: 7.609464908000064
inference time for step 16: 7.588756832999934
inference time for step 17: 7.581812891000027
inference time for step 18: 7.56099839500007
inference time f

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

And of the sons of Elizaphan; Shimri, and Jeiel: and of the sons of Asaph; Zechariah, and Mattaniah:	و چون قربانی های سوختنی تمام شد، پادشاه و جمیع حاضرین با وی رکوع کرده، سجده نمودند.	و پسران الیزایات، شمری و جیل: و پسران آزوف	2.242173863464727
All the pillars round about the court shall be filleted with silver; their hooks shall be of silver, and their sockets of brass.	«و تو بنی‌اسرائیل را امر فرما که روغن زیتون مصفی و کوبیده شده برای روشنایی نزد توبیاورند تا چراغها دائم روشن شود.در خیمه اجتماع، بیرون پرده‌ای که در برابر شهادت است، هارون و پسرانش از شام تا صبح، به حضورخداوند آن را درست کنند. و این برای بنی‌اسرائیل نسلا بعد نسل فریضه ابدی باشد.	تمام ستون های اطراف دربار با نقره پوشیده خواهند شد؛	0.017266482133721492
And Moses said unto them, If the children of Gad and the children of Reuben will pass with you over Jordan, every man armed to battle, before the LORD, and the land shall be subdued before you; then ye shall give them the land of Gilead for a possession:	وبنی جاد، دیبون و

In [ ]:
output_file_name = "translation_en-fa_bible_test_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### all dataset

In [ ]:
!nvidia-smi
!lscpu

Sat Jul  3 20:59:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    32W /  70W |   9560MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(bible_original_all, bible_translated_all, device, max_length=512, batch_size=128)

#original_text:31020, #translated_text:31020
#batch: 243
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 7.545222307000131
inference time for step 1: 7.5133224050000535
inference time for step 2: 7.544465235999951
inference time for step 3: 7.586130218000108
inference time for step 4: 7.584310578999975
inference time for step 5: 7.576429594000047
inference time for step 6: 7.569117514000027
inference time for step 7: 7.55096255400008
inference time for step 8: 7.546789387999979
inference time for step 9: 7.532956531000082
inference time for step 10: 7.533381243000122
inference time for step 11: 7.538904776999971
inference time for step 12: 7.548395226999901
inference time for step 13: 7.548108506000062
inference time for step 14: 7.552225691000103
inference time for step 15: 7.561022521000041
inference time for step 16: 7.561858830999881
inference time for step 17: 7.559065184000019
inference time for step 18: 7.571590088999983
inference tim

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خداوند آسمان و زمین را خلق کرد.	17.747405280050266
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و خالی بود و تاریکی بر چهره عمیق بود	12.490948835673722
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند می گوید: نور وجود دارد: و نور وجود داشت.	4.016138436407654
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خدا نور را دید و خوب دید: و خدا نور را از تاریکی جدا کرد	19.923405658137927
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید.

In [ ]:
output_file_name = "translation_en-fa_bible_all_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## translation_combined

In [8]:
!mkdir translation_combined_en_fa
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/translation_combined_en_fa/test.tsv -O translation_combined_en_fa/test.tsv
!ls translation_combined_en_fa

--2021-07-07 10:14:12--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/translation_combined_en_fa/test.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29348716 (28M) [text/plain]
Saving to: ‘translation_combined_en_fa/test.tsv’

translation_combine 100%[===================>]  27.99M  --.-KB/s    in 0.1s    

2021-07-07 10:14:13 (198 MB/s) - ‘translation_combined_en_fa/test.tsv’ saved [29348716/29348716]

test.tsv


### All combined test set

In [9]:
combined_original_test, combined_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv")
print(combined_original_test[0])
print(combined_translated_test[0])

test part:
 #original: 48123, #translation: 48123
In the beginning God created the heaven and the earth.
در ابتدا، خدا آسمانها و زمین را آفرید.


In [ ]:
!nvidia-smi
!lscpu

Sun Jul  4 03:57:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(combined_original_test, combined_translated_test, device, max_length=512, batch_size=128)

#original_text:48123, #translated_text:48123
#batch: 376
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 7.140410615999997
inference time for step 1: 6.944459954000024
inference time for step 2: 7.012005371999976
inference time for step 3: 7.043049408999991
inference time for step 4: 7.094806046999963
inference time for step 5: 7.120673291999992
inference time for step 6: 7.184457092999992
inference time for step 7: 7.256410141999993
inference time for step 8: 7.309968373000004
inference time for step 9: 7.376188813999988
inference time for step 10: 7.444738385999983
inference time for step 11: 7.515979589999972
inference time for step 12: 7.523403899000016
inference time for step 13: 7.546721940999987
inference time for step 14: 7.638839157999996
inference time for step 15: 7.721266261999972
inference time for step 16: 7.7978986529999474
inference time for step 17: 7.841218506000018
inference time for step 18: 7.898108101000048
inference ti

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خداوند آسمان و زمین را خلق کرد.	17.747405280050266
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و خالی بود و تاریکی بر چهره عمیق بود	12.490948835673722
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند می گوید: نور وجود دارد: و نور وجود داشت.	4.016138436407654
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خدا نور را دید و خوب دید: و خدا نور را از تاریکی جدا کرد	19.923405658137927
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید.

In [ ]:
output_file_name = "translation_en-fa_combined_all_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

In [10]:
!nvidia-smi
!lscpu

Wed Jul  7 10:14:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
evaluation_output = mt_model.mt5_evaluation(combined_original_test, combined_translated_test, device, max_length=512, split_reference='///', batch_size=128)

#original_text:48123, #translated_text:48123
#batch: 376
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 7.099174398000059
inference time for step 1: 6.9021900550000055
inference time for step 2: 6.941406848999918
inference time for step 3: 6.975670407000052
inference time for step 4: 7.008394536999958
inference time for step 5: 7.043930014000011
inference time for step 6: 7.0989856899999495
inference time for step 7: 7.155031703999953
inference time for step 8: 7.207082474999993
inference time for step 9: 7.256227201999991
inference time for step 10: 7.312034175000008
inference time for step 11: 7.356876759999977
inference time for step 12: 7.404770116999998
inference time for step 13: 7.478742409000006
inference time for step 14: 7.51199783200002
inference time for step 15: 7.477951915000062
inference time for step 16: 7.433592462999968
inference time for step 17: 7.404199921999975
inference time for step 18: 7.378155332999995
inference t

In [12]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خداوند آسمان و زمین را خلق کرد.	17.747405280050266
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و خالی بود و تاریکی بر چهره عمیق بود	12.490948835673722
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند می گوید: نور وجود دارد: و نور وجود داشت.	4.016138436407654
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خدا نور را دید و خوب دید: و خدا نور را از تاریکی جدا کرد	19.923405658137927
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید.

In [13]:
output_file_name = "translation_en-fa_combined_all_split-refs_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Bible subset

In [ ]:
bible_original_test, bible_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='bible_en_fa')
print(bible_original_test[0])
print(bible_translated_test[0])

test part:
 #original: 31020, #translation: 31020
In the beginning God created the heaven and the earth.
در ابتدا، خدا آسمانها و زمین را آفرید.


In [ ]:
!nvidia-smi
!lscpu

Sun Jul  4 04:49:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    33W /  70W |  11086MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(bible_original_test, bible_translated_test, device, max_length=512, batch_size=128)

#original_text:31020, #translated_text:31020
#batch: 243
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 7.802800196000135
inference time for step 1: 7.955733447000057
inference time for step 2: 8.157847744000264
inference time for step 3: 8.309127996999905
inference time for step 4: 8.169105281000157
inference time for step 5: 8.024447986999803
inference time for step 6: 7.9330983770000785
inference time for step 7: 7.84688072300014
inference time for step 8: 7.817898898999829
inference time for step 9: 7.812274018000153
inference time for step 10: 7.831825886999923
inference time for step 11: 7.864567994000026
inference time for step 12: 7.918188228999952
inference time for step 13: 7.975203237000187
inference time for step 14: 7.98115438800005
inference time for step 15: 7.961945340000057
inference time for step 16: 7.932368896999833
inference time for step 17: 7.911259715999677
inference time for step 18: 7.909181680000074
inference time

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خداوند آسمان و زمین را خلق کرد.	17.747405280050266
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و خالی بود و تاریکی بر چهره عمیق بود	12.490948835673722
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند می گوید: نور وجود دارد: و نور وجود داشت.	4.016138436407654
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خدا نور را دید و خوب دید: و خدا نور را از تاریکی جدا کرد	19.923405658137927
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید.

In [ ]:
output_file_name = "translation_en-fa_combined_bible_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Mizan subset

In [ ]:
mizan_original_test, mizan_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='mizan_test_en_fa')
print(mizan_original_test[0])
print(mizan_translated_test[0])

test part:
 #original: 10000, #translation: 10000
a sure sign with him of mental agitation.
این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.


In [ ]:
!nvidia-smi
!lscpu

Sun Jul  4 05:22:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    33W /  70W |   9942MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(mizan_original_test, mizan_translated_test, device, max_length=512, batch_size=128)

#original_text:10000, #translated_text:10000
#batch: 79
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 7.844372909000413
inference time for step 1: 7.936604374999661
inference time for step 2: 8.149338066999917
inference time for step 3: 8.176246910999907
inference time for step 4: 8.052928123999664
inference time for step 5: 7.971232068000063
inference time for step 6: 7.874580040000183
inference time for step 7: 7.8481937099995775
inference time for step 8: 7.821884663999299
inference time for step 9: 7.801606996999908
inference time for step 10: 7.804531277999558
inference time for step 11: 7.809996033999596
inference time for step 12: 7.838005997999971
inference time for step 13: 7.901662691999263
inference time for step 14: 7.936801858999388
inference time for step 15: 7.918399387000136
inference time for step 16: 7.8955519119999735
inference time for step 17: 7.894464601000436
inference time for step 18: 7.888795762999507
inference ti

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

a sure sign with him of mental agitation.	این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.	نشانه قطعی اضطراب ذهنی او.	1.8110800993753928
;Cesare, you are worried about this affair,; she said at last.	عاقبت گفت: سزار! تو از این موضوع نگران شده‌ای؛	سرانجام گفت: «سپار» تو نگران این موضوع هستی.	11.884631831419354
;I am very sorry you feel so despondent over it;	بسیار متأسفم که نسبت به آن احساس بی اعتمادی می‌کنی؛	متاسفم که این قدر از این موضوع ناراحتی	4.300847718252331
but I could decide only as seemed right to me.;	اما من فقط بدان علت نتوانستم تصمیم بگیرم که آن کار به نظرم صحیح رسید.	اما من فقط وقتی فکر می کنم که درست به نظر می رس	10.372689474549311
;It is not the affair,; he answered, sullenly; ;I know nothing about it,	مارتینی، با اخم، پاسخ داد: به خاطر آن موضوع نیست، من در آن باره چیزی نمی‌دانم؛	او با لحنی عبوس پاسخ داد: این قضیه نیست. من چیزی	10.052623571648706
and it probably is all right, once you have consented to go into it.	هنگامی که تو قبول می‌کنی در کاری شرکت جویی قطع

In [ ]:
output_file_name = "translation_en-fa_combined_mizan_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### qqp subset

In [ ]:
qqp_original_test, qqp_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='qqp_train_en_fa')
print(qqp_original_test[0])
print(qqp_translated_test[0])

test part:
 #original: 1104, #translation: 1104
How can I slowly lose weight?
چگونه می توانم کم کم وزن خود را کاهش دهم؟


In [ ]:
!nvidia-smi
!lscpu

Sun Jul  4 05:32:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    33W /  70W |   9750MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(qqp_original_test, qqp_translated_test, device, max_length=512, batch_size=128)

#original_text:1104, #translated_text:1104
#batch: 9
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 7.852938803000143
inference time for step 1: 7.885325796999496
inference time for step 2: 7.992284111000117
inference time for step 3: 7.983814266000081
inference time for step 4: 7.932326488000399
inference time for step 5: 7.874603588999889
inference time for step 6: 7.864364002999537
inference time for step 7: 7.842034858000261
inference time for step 8: 4.981762819999858
total inference time: 68.20945473499978
total inference time / #samples: 0.061783926390398355
BLEU Score: 19.97198782738506


In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

How can I slowly lose weight?	چگونه می توانم کم کم وزن خود را کاهش دهم؟	چطور می توانم به کندی وزنم را از دست بدهم؟	9.287528999566801
How does masturbation affect one's concentrating power?	چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟	استمناء چگونه بر قدرت تمرکز انسان تاثیر می گذارد؟	24.274588585366175
What does it mean when I dreamed of someone and said they're dying?	وقتی خواب کسی را دیدم و گفتم آنها در حال مرگ هستند ، چه معنایی دارد؟	معنی این چیست که من خواب کسی را دیدم و گفتم آنها در	44.40953195453769
How can I modify apk files?	چگونه می توانم فایل های apk را تغییر دهم؟	چطور می توانم پرونده های آپلیکیشن را اصلاح کنم؟	11.339582221952005
Where can I buy first copy shoes in mumbai?	از کجا می توانم کفش های کپی درجه اول را در بمبئی بخرم؟	کجا می توانم کفش های اولین نسخه را در بمبئی بخرم	39.12360608928473
What is the step by step procedure to apply for Australian PR?	روش مرحله به مرحله برای درخواست شهروندی دایم استرالیا چیست؟	روند قدم به قدم برای استفاده از تبلیغات استرالیایی چیست	4.990

In [ ]:
output_file_name = "translation_en-fa_combined_qqp_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Quran subset

In [14]:
quran_original_test, quran_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='quran_en_fa')
print(quran_original_test[0])
print(quran_translated_test[0])

test part:
 #original: 5999, #translation: 5999
In the name of Allah, Most Gracious, Most Merciful.
به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌


In [ ]:
!nvidia-smi
!lscpu

Sun Jul  4 05:33:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    44W /  70W |   9560MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(quran_original_test, quran_translated_test, device, max_length=512, batch_size=128)

#original_text:5999, #translated_text:5999
#batch: 47
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 7.914699657999336
inference time for step 1: 7.926704975000575
inference time for step 2: 7.9509270970002035
inference time for step 3: 7.985907985000267
inference time for step 4: 7.985799926999789
inference time for step 5: 8.001785831000234
inference time for step 6: 7.956813791000059
inference time for step 7: 7.944109811000089
inference time for step 8: 7.950598950999847
inference time for step 9: 7.925366749999739
inference time for step 10: 7.9184883679999984
inference time for step 11: 7.92731756899957
inference time for step 12: 7.91990035300023
inference time for step 13: 7.934527632999561
inference time for step 14: 7.885443799999848
inference time for step 15: 7.921507068999745
inference time for step 16: 7.95868915400024
inference time for step 17: 7.946426542000154
inference time for step 18: 7.951819176999379
inference time fo

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the name of Allah, Most Gracious, Most Merciful.	به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌	به نام الله، گرامی ترین و قابل ترحیم ترین.	0.0010083832740421274
Praise be to Allah, the Cherisher and Sustainer of the worlds;	همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///ستایش خدا را که پروردگار جهانیان است.///ستایش خداى را که پروردگار جهانیان است///ستایش خدایى را که پروردگار جهانیان،///سپاس و ستایش مخصوص خداوندى است که پروردگار جهانیان است.///ستایش خدای را که پروردگار جهانیان است.///سپاس خداوند را که پروردگار جهانیان است‌///ستایش مخصوص خداوندی است که پروردگار جهانیان ا

In [ ]:
output_file_name = "translation_en-fa_combined_quran_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

In [15]:
!nvidia-smi
!lscpu

Wed Jul  7 11:12:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    26W /  70W |  10446MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
evaluation_output = mt_model.mt5_evaluation(quran_original_test, quran_translated_test, device, max_length=512, split_reference='///', batch_size=128)

#original_text:5999, #translated_text:5999
#batch: 47
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 6.986514386999261
inference time for step 1: 6.938277708999522
inference time for step 2: 6.960767912999472
inference time for step 3: 6.9831922240000495
inference time for step 4: 7.002279919999637
inference time for step 5: 7.0402758019999965
inference time for step 6: 7.056938359999549
inference time for step 7: 7.0674754969995774
inference time for step 8: 7.082891015000314
inference time for step 9: 7.1032416269999885
inference time for step 10: 7.118814649999877
inference time for step 11: 7.146834607000528
inference time for step 12: 7.1630315539996445
inference time for step 13: 7.164687112000138
inference time for step 14: 7.155742240000109
inference time for step 15: 7.185219126999982
inference time for step 16: 7.205238197000654
inference time for step 17: 7.210406518999662
inference time for step 18: 7.218893438000123
inference 

In [17]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the name of Allah, Most Gracious, Most Merciful.	به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌	به نام الله، گرامی ترین و قابل ترحیم ترین.	9.980099403873663
Praise be to Allah, the Cherisher and Sustainer of the worlds;	همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///ستایش خدا را که پروردگار جهانیان است.///ستایش خداى را که پروردگار جهانیان است///ستایش خدایى را که پروردگار جهانیان،///سپاس و ستایش مخصوص خداوندى است که پروردگار جهانیان است.///ستایش خدای را که پروردگار جهانیان است.///سپاس خداوند را که پروردگار جهانیان است‌///ستایش مخصوص خداوندی است که پروردگار جهانیان است./

In [18]:
output_file_name = "translation_en-fa_combined_quran_split-refs_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## QQP Dataset

## TEP Dataset


## OPUS Dataset

## Global Voice Dataset